In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df=pd.read_csv("property_listing_data_in_Bangladesh.csv")
df.head(5)
#print(df['type'].loc[df.index[0]])

,title,beds,bath,area,adress,type,purpose,flooPlan,url,lastUpdated,price
0,Eminent Apartment Of 2200 Sq Ft Is Vacant For ...,3,4,"2,200 sqft","Block A, Bashundhara R-A, Dhaka",Apartment,For Rent,https://images-cdn.bproperty.com/thumbnails/10...,https://www.bproperty.com/en/property/details-...,"August 13, 2022",50 Thousand
1,"Apartment Ready To Rent In South Khulshi, Near...",3,4,"1,400 sqft","South Khulshi, Khulshi, Chattogram",Apartment,For Rent,https://images-cdn.bproperty.com/thumbnails/44...,https://www.bproperty.com/en/property/details-...,"January 25, 2022",30 Thousand
2,"Smartly priced 1950 SQ FT apartment, that you ...",3,4,"1,950 sqft","Block F, Bashundhara R-A, Dhaka",Apartment,For Rent,https://images-cdn.bproperty.com/thumbnails/11...,https://www.bproperty.com/en/property/details-...,"February 22, 2023",30 Thousand
3,2000 Sq Ft Residential Apartment Is Up For Ren...,3,3,"2,000 sqft","Sector 9, Uttara, Dhaka",Apartment,For Rent,https://images-cdn.bproperty.com/thumbnails/14...,https://www.bproperty.com/en/property/details-...,"October 28, 2021",35 Thousand
4,Strongly Structured This 1650 Sq. ft Apartment...,3,4,"1,650 sqft","Block I, Bashundhara R-A, Dhaka",Apartment,For Rent,https://images-cdn.bproperty.com/thumbnails/10...,https://www.bproperty.com/en/property/details-...,"February 19, 2023",25 Thousand


In [3]:
n=df.shape[1]
#df=df[df['type'].index!="Apartment"]
#removing Building and Duplex type
df.drop(df.loc[df['type']==0].index, inplace=True)
#removing extra string type data
df=df.drop(['title','adress','type','purpose','flooPlan','url','lastUpdated'],axis=1)
print(df.head(5))
print(df.shape)

  beds bath        area        price
0   3    4   2,200 sqft  50 Thousand
1   3    4   1,400 sqft  30 Thousand
2   3    4   1,950 sqft  30 Thousand
3   3    3   2,000 sqft  35 Thousand
4   3    4   1,650 sqft  25 Thousand
(7557, 4)


In [4]:
df.dtypes

beds     object
bath     object
area     object
price    object
dtype: object

In [5]:
#Removing Lakh and thousand
n,m=df.shape
ts="0123456789"
for i in range(n):
    s=df.beds[i]
    for j in s:
        if(j not in ts):
            s=s.replace(j,"")
    df.beds[i]=int(s)
    s=df.bath[i]
    #print(s)
    for j in s:
        if(j not in ts):
            s=s.replace(j,"")
    df.bath[i]=int(s)
    #print(s)
    s=df.area[i]
    for j in s:
        if(j not in ts):
            s=s.replace(j,"")
    df.area[i]=int(s)
    #print(s)
    s=df.price[i]
    st="Thousand"
    sl="Lakh"
    if(st in s):
        a=1000
    if(sl in s):
        a=100000
    for j in s:
        if(j not in ts):
            s=s.replace(j,"")
    df.price[i]=int(s)*a
    
#df['beds']=pd.to_int(df['beds'])
df.head(10)

,beds,bath,area,price
0,3,4,2200,50000
1,3,4,1400,30000
2,3,4,1950,30000
3,3,3,2000,35000
4,3,4,1650,25000
5,5,5,3400,1100000
6,3,3,1600,35000
7,3,3,1250,23000
8,3,4,2150,40000
9,3,3,1250,23000


In [6]:
marea=np.mean(df['area'])   #Z scor normalization
stdarea=np.std(df['area'])
df['area']=(df['area']-marea)/stdarea
print(marea)
print(stdarea)
print(f"DF Area\n{df.head(4)}")

1372.0250099245732
956.6929589970397
DF Area
  beds bath      area  price
0    3    4  0.865455  50000
1    3    4  0.029241  30000
2    3    4  0.604138  30000
3    3    3  0.656402  35000


In [7]:
#mprice=np.mean(df['price'])         #Z scor normalization
#stdprice=np.std(df['price'])
#df['price']=(df['price']-mprice)/stdprice
#print(mprice)
#print(stdprice)
#print(f"DF Price\n{df.head(20)}")

In [8]:
def Predict(W,b,X):
    m,_=X.shape
    f_wb=np.zeros(m)
    f_wb=f_wb.reshape(m,1)
    
    for i in range(m):
        f_wb[i]=np.dot(X.iloc[i],W)+b
    
    #f_wb=f_wb.reshape((1,m))
    return f_wb


In [9]:
def Cost_Function(W,b,X,Y):
    f_wb=Predict(W,b,X)
    m,_=X.shape
    cost=0
    #for i in range(m):
    cost=np.sum((f_wb-Y)**2)
    totcost=cost/(2*m)
    return totcost


In [10]:
def Gradient(W,b,X,Y):
    
    m,n=X.shape
    db=0.
    dw=np.zeros((n,1))
    f_wb=Predict(W,b,X)
    #lit=['beds','bath','area']
    db=np.sum(f_wb-Y)
    dw=np.dot(X.T,(f_wb-Y))
    #for i in range(m):
        #W=W-alpha*((1/m)*np.sum(f_wb-Y),axis=0)
     #   for j in range(n):
      #      dw[j]+=(f_wb[i]-Y.iloc[i])*X.iloc[i, j]
       # db+=(f_wb[i]-Y.iloc[i])
    db=db/m
    dw=dw/m
    
    return dw,db

In [11]:
def Gradient_Descent(W_in,b_in,X,Y,X_v,Y_v,num_iteration,alpha):
    
    #J_history=[]
    W=W_in
    b=b_in
    
    for i in range(num_iteration):
        dw,db=Gradient(W,b,X,Y)
        
        W=W-alpha*dw
        b=b-alpha*db
        
        #J_history.append(Cost_Function(W,b,X,Y))
        train_loss=Cost_Function(W,b,X,Y)
        valid_loss=Cost_Function(W,b,X_v,Y_v)
        print(f"Iteration {i:4d}  ->  Training Loss: {train_loss.item():8.2f}     Validation Loss: {valid_loss.item():0.2f}")
            
            
    return W,b

In [12]:
X=df[['beds','bath','area']]
Y=df[['price']]
print(X.head(5))
print(Y.head(5))

  beds bath      area
0    3    4  0.865455
1    3    4  0.029241
2    3    4  0.604138
3    3    3  0.656402
4    3    4  0.290558
   price
0  50000
1  30000
2  30000
3  35000
4  25000


In [13]:
from sklearn.model_selection import train_test_split

X.rename(columns={X.columns[0]: 0},inplace=True)
X.rename(columns={X.columns[1]: 1},inplace=True)
X.rename(columns={X.columns[2]: 2},inplace=True)

X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.98,random_state=42)
X_valid,X_test,Y_valid,Y_test=train_test_split(X_test,Y_test,test_size=0.97,random_state=42)
X_test,x,Y_test,y=train_test_split(X_test,Y_test,test_size=0.95,random_state=42)


print(X.head(10))
print(X_train.shape)
print(Y_train.shape)
print(X_valid.shape)
print(Y_valid.shape)
print(X_test.shape)
print(Y_test.shape)


   0  1         2
0  3  4  0.865455
1  3  4  0.029241
2  3  4  0.604138
3  3  3  0.656402
4  3  4  0.290558
5  5  5  2.119776
6  3  3  0.238295
7  3  3 -0.127549
8  3  4  0.813192
9  3  3 -0.127549
(151, 3)
(151, 1)
(222, 3)
(222, 1)
(359, 3)
(359, 1)


C:\Users\hp\AppData\Local\Temp\ipykernel_11040\1477788151.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.rename(columns={X.columns[0]: 0},inplace=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_11040\1477788151.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.rename(columns={X.columns[1]: 1},inplace=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_11040\1477788151.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.rename(co

In [14]:
# initialize parameters
#initial_w = np.zeros(X_train.shape[0])
initial_w = np.random.rand(X_train.shape[1],1)
#initial_w =initial_w.reshape(X_train.shape[0],1)
initial_b = 5.0
print(initial_w)

iterations = 1000
alpha = 0.01


w_final, b_final = Gradient_Descent(initial_w, initial_b, X_train, Y_train, X_valid,Y_valid, iterations,alpha )
print(f"b,w found by gradient descent: {b_final.item():0.2f}, {w_final}")


[[0.81574482]
 [0.91558575]
 [0.68562294]]
Iteration    0  ->  Training Loss: 97863759676.63     Validation Loss: 64996670156.82
Iteration    1  ->  Training Loss: 94627336790.37     Validation Loss: 62643509356.12
Iteration    2  ->  Training Loss: 92236707472.69     Validation Loss: 61148347964.62
Iteration    3  ->  Training Loss: 90423891517.05     Validation Loss: 60213529749.37
Iteration    4  ->  Training Loss: 89006719152.54     Validation Loss: 59640781696.02
Iteration    5  ->  Training Loss: 87861338943.19     Validation Loss: 59298741594.57
Iteration    6  ->  Training Loss: 86903533888.62     Validation Loss: 59100984972.12
Iteration    7  ->  Training Loss: 86076023660.41     Validation Loss: 58991173531.50
Iteration    8  ->  Training Loss: 85339835072.09     Validation Loss: 58933031418.80
Iteration    9  ->  Training Loss: 84668437358.71     Validation Loss: 58903592204.57
Iteration   10  ->  Training Loss: 84043756446.95     Validation Loss: 58888659817.95
Iteration  

Iteration   96  ->  Training Loss: 58386802510.23     Validation Loss: 67046521226.07
Iteration   97  ->  Training Loss: 58246037517.26     Validation Loss: 67226544321.11
Iteration   98  ->  Training Loss: 58107383189.98     Validation Loss: 67407054207.14
Iteration   99  ->  Training Loss: 57970805953.28     Validation Loss: 67588016701.77
Iteration  100  ->  Training Loss: 57836272768.05     Validation Loss: 67769398373.15
Iteration  101  ->  Training Loss: 57703751122.70     Validation Loss: 67951166526.40
Iteration  102  ->  Training Loss: 57573209024.67     Validation Loss: 68133289190.31
Iteration  103  ->  Training Loss: 57444614992.16     Validation Loss: 68315735104.15
Iteration  104  ->  Training Loss: 57317938045.99     Validation Loss: 68498473704.83
Iteration  105  ->  Training Loss: 57193147701.55     Validation Loss: 68681475114.17
Iteration  106  ->  Training Loss: 57070213960.90     Validation Loss: 68864710126.49
Iteration  107  ->  Training Loss: 56949107305.04     

Iteration  194  ->  Training Loss: 50990548596.61     Validation Loss: 83789943250.05
Iteration  195  ->  Training Loss: 50955044338.75     Validation Loss: 83932620802.51
Iteration  196  ->  Training Loss: 50919981885.59     Validation Loss: 84074562445.57
Iteration  197  ->  Training Loss: 50885354296.90     Validation Loss: 84215767573.81
Iteration  198  ->  Training Loss: 50851154743.11     Validation Loss: 84356235686.33
Iteration  199  ->  Training Loss: 50817376503.55     Validation Loss: 84495966384.27
Iteration  200  ->  Training Loss: 50784012964.71     Validation Loss: 84634959368.45
Iteration  201  ->  Training Loss: 50751057618.50     Validation Loss: 84773214437.10
Iteration  202  ->  Training Loss: 50718504060.63     Validation Loss: 84910731483.51
Iteration  203  ->  Training Loss: 50686345988.88     Validation Loss: 85047510493.84
Iteration  204  ->  Training Loss: 50654577201.54     Validation Loss: 85183551544.90
Iteration  205  ->  Training Loss: 50623191595.73     

Iteration  293  ->  Training Loss: 48846412674.67     Validation Loss: 94511263496.38
Iteration  294  ->  Training Loss: 48833289049.73     Validation Loss: 94587940383.25
Iteration  295  ->  Training Loss: 48820260429.46     Validation Loss: 94664074383.11
Iteration  296  ->  Training Loss: 48807325396.03     Validation Loss: 94739668355.01
Iteration  297  ->  Training Loss: 48794482554.08     Validation Loss: 94814725155.89
Iteration  298  ->  Training Loss: 48781730530.41     Validation Loss: 94889247640.25
Iteration  299  ->  Training Loss: 48769067973.59     Validation Loss: 94963238659.86
Iteration  300  ->  Training Loss: 48756493553.61     Validation Loss: 95036701063.47
Iteration  301  ->  Training Loss: 48744005961.59     Validation Loss: 95109637696.55
Iteration  302  ->  Training Loss: 48731603909.38     Validation Loss: 95182051400.96
Iteration  303  ->  Training Loss: 48719286129.26     Validation Loss: 95253945014.72
Iteration  304  ->  Training Loss: 48707051373.60     

Iteration  389  ->  Training Loss: 47874360921.86     Validation Loss: 99785000281.32
Iteration  390  ->  Training Loss: 47866194271.33     Validation Loss: 99821631486.37
Iteration  391  ->  Training Loss: 47858052367.84     Validation Loss: 99857959130.60
Iteration  392  ->  Training Loss: 47849934902.61     Validation Loss: 99893985200.05
Iteration  393  ->  Training Loss: 47841841571.67     Validation Loss: 99929711669.62
Iteration  394  ->  Training Loss: 47833772075.79     Validation Loss: 99965140503.03
Iteration  395  ->  Training Loss: 47825726120.37     Validation Loss: 100000273652.90
Iteration  396  ->  Training Loss: 47817703415.44     Validation Loss: 100035113060.69
Iteration  397  ->  Training Loss: 47809703675.49     Validation Loss: 100069660656.82
Iteration  398  ->  Training Loss: 47801726619.50     Validation Loss: 100103918360.60
Iteration  399  ->  Training Loss: 47793771970.77     Validation Loss: 100137888080.32
Iteration  400  ->  Training Loss: 47785839456.93

...

Iteration  585  ->  Training Loss: 46537827966.42     Validation Loss: 103031561368.53
Iteration  586  ->  Training Loss: 46531808815.66     Validation Loss: 103034993721.44
Iteration  587  ->  Training Loss: 46525795014.76     Validation Loss: 103038344461.19
Iteration  588  ->  Training Loss: 46519786545.11     Validation Loss: 103041614116.32
Iteration  589  ->  Training Loss: 46513783388.30     Validation Loss: 103044803211.57
Iteration  590  ->  Training Loss: 46507785526.15     Validation Loss: 103047912267.88
Iteration  591  ->  Training Loss: 46501792940.68     Validation Loss: 103050941802.46
Iteration  592  ->  Training Loss: 46495805614.12     Validation Loss: 103053892328.78
Iteration  593  ->  Training Loss: 46489823528.90     Validation Loss: 103056764356.59
Iteration  594  ->  Training Loss: 46483846667.64     Validation Loss: 103059558392.01
Iteration  595  ->  Training Loss: 46477875013.19     Validation Loss: 103062274937.46
Iteration  596  ->  Training Loss: 46471908

Iteration  683  ->  Training Loss: 45971150137.06     Validation Loss: 103048326843.14
Iteration  684  ->  Training Loss: 45965590601.15     Validation Loss: 103045781090.64
Iteration  685  ->  Training Loss: 45960035261.08     Validation Loss: 103043190977.09
Iteration  686  ->  Training Loss: 45954484109.43     Validation Loss: 103040556763.39
Iteration  687  ->  Training Loss: 45948937138.83     Validation Loss: 103037878708.59
Iteration  688  ->  Training Loss: 45943394341.95     Validation Loss: 103035157069.94
Iteration  689  ->  Training Loss: 45937855711.52     Validation Loss: 103032392102.82
Iteration  690  ->  Training Loss: 45932321240.32     Validation Loss: 103029584060.83
Iteration  691  ->  Training Loss: 45926790921.18     Validation Loss: 103026733195.78
Iteration  692  ->  Training Loss: 45921264746.97     Validation Loss: 103023839757.69
Iteration  693  ->  Training Loss: 45915742710.61     Validation Loss: 103020903994.81
Iteration  694  ->  Training Loss: 45910224

Iteration  778  ->  Training Loss: 45460832699.75     Validation Loss: 102639359627.65
Iteration  779  ->  Training Loss: 45455644046.33     Validation Loss: 102633547521.04
Iteration  780  ->  Training Loss: 45450459043.88     Validation Loss: 102627709221.14
Iteration  781  ->  Training Loss: 45445277687.82     Validation Loss: 102621844863.19
Iteration  782  ->  Training Loss: 45440099973.63     Validation Loss: 102615954581.52
Iteration  783  ->  Training Loss: 45434925896.77     Validation Loss: 102610038509.59
Iteration  784  ->  Training Loss: 45429755452.76     Validation Loss: 102604096779.97
Iteration  785  ->  Training Loss: 45424588637.09     Validation Loss: 102598129524.33
Iteration  786  ->  Training Loss: 45419425445.29     Validation Loss: 102592136873.50
Iteration  787  ->  Training Loss: 45414265872.90     Validation Loss: 102586118957.42
Iteration  788  ->  Training Loss: 45409109915.49     Validation Loss: 102580075905.19
Iteration  789  ->  Training Loss: 45403957

Iteration  875  ->  Training Loss: 44973909581.89     Validation Loss: 101970848776.05
Iteration  876  ->  Training Loss: 44969055869.65     Validation Loss: 101963013752.22
Iteration  877  ->  Training Loss: 44964205426.09     Validation Loss: 101955162317.57
Iteration  878  ->  Training Loss: 44959358247.75     Validation Loss: 101947294545.41
Iteration  879  ->  Training Loss: 44954514331.19     Validation Loss: 101939410508.66
Iteration  880  ->  Training Loss: 44949673672.97     Validation Loss: 101931510279.80
Iteration  881  ->  Training Loss: 44944836269.66     Validation Loss: 101923593930.86
Iteration  882  ->  Training Loss: 44940002117.83     Validation Loss: 101915661533.48
Iteration  883  ->  Training Loss: 44935171214.07     Validation Loss: 101907713158.87
Iteration  884  ->  Training Loss: 44930343554.98     Validation Loss: 101899748877.83
Iteration  885  ->  Training Loss: 44925519137.15     Validation Loss: 101891768760.73
Iteration  886  ->  Training Loss: 44920697

Iteration  975  ->  Training Loss: 44504187697.80     Validation Loss: 101116494622.87
Iteration  976  ->  Training Loss: 44499644880.43     Validation Loss: 101107322701.36
Iteration  977  ->  Training Loss: 44495105019.50     Validation Loss: 101098140001.74
Iteration  978  ->  Training Loss: 44490568112.20     Validation Loss: 101088946566.72
Iteration  979  ->  Training Loss: 44486034155.69     Validation Loss: 101079742438.79
Iteration  980  ->  Training Loss: 44481503147.16     Validation Loss: 101070527660.27
Iteration  981  ->  Training Loss: 44476975083.79     Validation Loss: 101061302273.24
Iteration  982  ->  Training Loss: 44472449962.78     Validation Loss: 101052066319.60
Iteration  983  ->  Training Loss: 44467927781.32     Validation Loss: 101042819841.03
Iteration  984  ->  Training Loss: 44463408536.60     Validation Loss: 101033562879.03
Iteration  985  ->  Training Loss: 44458892225.84     Validation Loss: 101024295474.87
Iteration  986  ->  Training Loss: 44454378

In [15]:
Y_test_predict=Predict(w_final,b_final,X_test)
test_loss=Cost_Function(w_final,b_final,X_test,Y_test)
print(f"Test Loss: {float(test_loss)} ")
print(Y_test_predict.shape)

Test Loss: 398095065324.66504 
(359, 1)


C:\Users\hp\AppData\Local\Temp\ipykernel_11040\3017757641.py:3: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  print(f"Test Loss: {float(test_loss)} ")


In [16]:
test_mean=np.mean(Y_test)
mse_total= np.sum((Y_test-test_mean)**2)
mse_res = np.sum((Y_test-Y_test_predict)**2)
R_Squared=1-(mse_res/mse_total)
print(f"R-Squared: {float(R_Squared)} ")

R-Squared: 19.873877798866303 


C:\Users\hp\AppData\Local\Temp\ipykernel_11040\453783937.py:5: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  print(f"R-Squared: {float(R_Squared*100)} ")
